[//]: # (Copyright c 2021, Alessandro Abate, Daniele Ahmed, Alec Edwards, Mirco Giacobbe, Andrea Peruffo)
[//]: # (All rights reserved.)
[//]: # (This source code is licensed under the BSD-style license found in the)
[//]: # (LICENSE file in the root directory of this source tree. )

# 🐚 Welcome to FOSSIL!

            ..\\\\
         ..\\\\  ,\
    --===)))))))   )
         ''////  `/
            ''////  

                                                                                                                   
## Overview
This tool uses neural network templates alongside the CEGIS architecture to automatically synthesise sound Lyapunov functions of Barrier Certificates for any N-dimensional system. 

To use this tool please change the python code in the corresponding cells to input your required dynamical system and domain, then use the correspoding cell to synthise a [Lyapunov function](#lyap-synth) of [Barrier function](#barrier-synth) respectively.

If you would like access to more settings, such as , please use the [advanced-playground](FOSSIL-advanced-playground.ipynb). To see benchmarks and further examples refer to [benchmarks](FOSSIL-benchmarks.ipynb).

In [ ]:
# % Imports
import sys
sys.path.append('..')
from experiments.playground_utils import *
#sys.path.append(".")

### Synthesising Lyapunov Functions

First, we demonstrate synthesising Lyapunov functions. Consider the following dynamical system:

$$ \begin{cases} \dot{x} = -x + xy \\ \dot{y} = - y \end{cases}$$


This system can be instantied as follows, with $x_0 = x $ and $x_1 = y$:


In [ ]:

N_Dimensions = 2

x0, x1 = initialise_states(N_Dimensions)

dynamics = [
    -x0 + x0 * x1,
    -x1
]

# Note, any arbitrary variables can be used here, but internally they will be treated as x_0,...,x_n and the
# returned function will be a function of these parameters.

This system has an equilibrium point at the origin $(x_0, x_1) = (0, 0)$. We seek to synthesis a Lyapunov function to serve as a certificate of stability of this equilibrium point. This Lyapunov function will be validated in a region around the equilibrium point. This domain is defined as:

$$ \mathcal{D}  = \{(x_0, x_1): x_0^2 + x_1^2 < r^2 \} $$

for some user-set parameter r. To constrain the domain to a torus shape (i.e., excluding a smaller hypershere around the origin), use the keyword arguement INNER_RADIUS. 
<a id='lyap-synth'></a>

In [ ]:
r = 10
mode = PrimerMode.LYAPUNOV
numerical_V, symbolic_V = synthesise(dynamics, mode, OUTER_RADIUS=r, CEGIS_PARAMETERS={CegisConfig.SEED_AND_SPEED.k: True})

In [ ]:
# The symbolic function can be printed using print_f
print_f(symbolic_V)

### Barrier Cerificates


We now demonstrate synthesising a barrier certificate. Consider the following dynamical system:

$$\begin{cases}
\dot{x} = y \\
\dot{y} = - 2400y - 576x
\end{cases}$$

We can initialise this dynamical system as follows, with with $x_0 = x $ and $x_1 = y$.

In [ ]:
N_Dimensions = 2

x0, x1= initialise_states(N_Dimensions)

dynamics = [
    x1,
    -2400*x1 - 576*x0
]

#### Domain Initialisation
We define the overall domain as 
$$\begin{align*}
& X = \{x_0^2 + x_1^2 \leq 4\}\\ 
& X_0 = \{(x_0 - 1)^2+(x_1 - 1)^2 \leq 0.25 \} \\ 
& X_u  = \{ (x_0 + 2)^2+(x_1 + 2)^2 \leq 0.16 \}
\end{align*}$$


This can be initialised as follows:

In [ ]:
xd = Sphere([0, 0], 2)
xi = Sphere([1, 1], 0.5)
xu = Sphere([-2, -2], 0.4)

#Hyper cuboids can be generated using Rectangle([lower_bounds], [upper_bounds])

A barrier function is then synthesised as:
<a id='barrier-synth'></a>

In [ ]:
verifier_type = VerifierType.DREAL
mode = PrimerMode.BARRIER

numerical_B, symbolic_B, = synthesise(dynamics, mode, XD=xd, XI=xi, XU=xu, 
                       CEGIS_PARAMETERS={CegisConfig.VERIFIER.k:VerifierType.DREAL, 
                                         CegisConfig.ACTIVATION.k: [ActivationType.LINEAR] })
